In [6]:
import os
import sys
import glob
import json
from collections import defaultdict

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import pandas as pd

In [7]:
log_files = glob.glob(os.path.join(os.getcwd(), "data", "*.log"))
log_files

['c:\\Users\\milan\\Repository\\music_signal_detector\\notebook\\data\\loud_music_max_gain.log',
 'c:\\Users\\milan\\Repository\\music_signal_detector\\notebook\\data\\loud_music_nnormal_gain.log',
 'c:\\Users\\milan\\Repository\\music_signal_detector\\notebook\\data\\no_signal_max_gain.log',
 'c:\\Users\\milan\\Repository\\music_signal_detector\\notebook\\data\\no_signal_normal_gain.log',
 'c:\\Users\\milan\\Repository\\music_signal_detector\\notebook\\data\\quite_music_max_gain.log',
 'c:\\Users\\milan\\Repository\\music_signal_detector\\notebook\\data\\quite_music_normal_gain.log',
 'c:\\Users\\milan\\Repository\\music_signal_detector\\notebook\\data\\silence_max_gain.log',
 'c:\\Users\\milan\\Repository\\music_signal_detector\\notebook\\data\\silence_normal_gain.log']

In [8]:
def parse_log_file(log_file):
    with open(log_file, 'r') as f:
        lines = f.readlines()
        
    lines = lines[2:-1] # first two lines are headers, last line might be incomplete

    data = defaultdict(list)
    for line in lines:
        line = line.strip()
        if not line.startswith(">"):\
            continue
        line = line[1:] # drop >
        line_parts = line.split(",")
        for part in line_parts:
            name, value = part.split(":")
            value = float(value)
            data[name].append(value)

    # print("Parsed data:")
    # for key, values in data.items():
    #     print(f"{key}: {values} (size: {len(values)})")

    df = pd.DataFrame(data)
    return df

all_df = {}
for log_file in log_files:
    print(f"Processing file: {log_file}")
    df = parse_log_file(log_file)
    all_df[os.path.basename(log_file)] = df

Processing file: c:\Users\milan\Repository\music_signal_detector\notebook\data\loud_music_max_gain.log
Processing file: c:\Users\milan\Repository\music_signal_detector\notebook\data\loud_music_nnormal_gain.log
Processing file: c:\Users\milan\Repository\music_signal_detector\notebook\data\no_signal_max_gain.log
Processing file: c:\Users\milan\Repository\music_signal_detector\notebook\data\no_signal_normal_gain.log
Processing file: c:\Users\milan\Repository\music_signal_detector\notebook\data\quite_music_max_gain.log
Processing file: c:\Users\milan\Repository\music_signal_detector\notebook\data\quite_music_normal_gain.log
Processing file: c:\Users\milan\Repository\music_signal_detector\notebook\data\silence_max_gain.log
Processing file: c:\Users\milan\Repository\music_signal_detector\notebook\data\silence_normal_gain.log


In [9]:
def compute_rolling_statistics(df, column='a0', window_size=25):
    """Compute rolling statistics for signal analysis"""
    # Make a copy to avoid modifying the original dataframe
    result = df.copy()
    
    # Compute rolling variance
    result['rolling_var'] = df[column].rolling(window=window_size).var()

    # Compute variance of variance (using another rolling window)
    result['var_of_var'] = result['rolling_var'].rolling(window=window_size).var()

    result['var_of_var_avg'] = result['var_of_var'].rolling(window=window_size).mean()
    
    # Drop NaN values from calculations
    result = result[['rolling_var', 'var_of_var', 'var_of_var_avg', 'a0']].dropna()
    
    return result

# Compute features for each dataset
analysis_results = {}
for filename, df in all_df.items():
    if "max_gain" in filename:
        continue

    statistics = compute_rolling_statistics(df)
    analysis_results[filename] = statistics

    # Visualize all statistics columns in a single plot
    fig = go.Figure()
    columns_to_visualize = ['rolling_var', 'var_of_var', "var_of_var_avg"]
    
    for column in columns_to_visualize:
        fig.add_trace(go.Scatter(
            x=statistics.index,
            y=statistics[column],
            mode='lines',
            name=f"{column} ({filename})"
        ))
    
    # Update layout
    fig.update_layout(
        title=f"Statistics over Time for {filename}",
        xaxis_title="Index",
        yaxis_title="Value",
        legend_title="Statistics"
    )
    
    fig.show()

    
    